# Notebook 5 : Web scraping

In [ ]:
# Décommenter la ligne suivante pour installer lxml (nécessaire pour read_html)
# %pip install lxml

In [ ]:
import pandas as pd
import requests

from bs4 import BeautifulSoup

## Bonsaïs

Le site [Umi Zen Bonsai](https://umizenbonsai.com/) est une boutique de vente en ligne dédiée aux bonsaïs. Les conifères sont disponible sur la page web [https://umizenbonsai.com/shop/bonsai/coniferes/](https://umizenbonsai.com/shop/bonsai/coniferes/). Comme beaucoup d'autres sites, l'information est organisée en blocs dans lesquels il est possible de récupérer des données.

Pour scraper ce type de site, le processus consiste à capturer les blocs dans un premier temps, puis à en extraire les données.

1. Récupérer le contenu de la page avec `requests` et passer le résultat au parser de `BeautifulSoup`.

2. Écrire un sélecteur CSS pour capturer les éléments `li` qui contiennent les blocs correspondants aux bonsaïs. Vérifier sur le site que le nombre de bonsaïs affichés correspond.

3. Écrire une fonction qui prend un bloc de la liste précédente et retourne un tuple contenant le nom, le prix et le lien de description du bonsaï.

4. Utiliser les deux questions précédentes pour construire un dataframe contenant les données des bonsaïs.

5. (*Bonus*) Écrire une fonction pour récupérer la provenance, le feuilage et les dimension du bonsaï à partir du lien de description. Utiliser cette fonction pour ajouter des colonnes au dataframe précédent

## Trampoline

Le trampoline est un sport olympique depuis les jeux de Sydney en 2000. La page suivante contient les listes des hommes et des femmes ayant obtenu une médaille olympique dans cette discipline :
[https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline](https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline)

Un tableau est contenu dans un élément `table` avec des balises pour les lignes `tr`, pour les colonnes `th`, pour les cellules `td`, ... Cela peut être fastidieux à scraper et très répétitif. Heureusement, Pandas propose la fonction `read_html` pour récupérer des tableaux sous forme de dataframes à partir d'une page web.

1. Utiliser la fonction `read_html` de Pandas sur la page des médaillés olympiques au trampoline. Combien de dataframes sont récupérés ?

2. Extraire de la liste précédente les dataframes des médailles masculines et féminines.

3. À partir de ces dataframes, compter combien chaque pays a reçu de médailles d'or, d'argent et de bronze.

4. (*Bonus*) Construire un dataframe contenant, pour chaque pays, le nombre de médailles d'or, d'argent et de bronze ainsi que le nombre total de médailles. Classer ce dataframe dans l'ordre usuel en fonction d'abord du nombre de médailles d'or, puis du nombre de médailles d'argent et enfin du nombre de médailles de bronze. Comparer le résultat avec le tableau des médailles sur la page [https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques](https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques).

## Cate Blanchett

Dans le cours, nous avons essayé de trouver avec quels acteurs Cate Blanchett a joué le plus au cours des années 2000. Pour cela, nous avons récupéré la liste des pages Wikipedia des films où elle tient un rôle avec le code suivant :

In [ ]:
url_wikipedia = "https://fr.wikipedia.org"
url_blanchett = url_wikipedia + "/wiki/Cate_Blanchett"

r_blanchett = requests.get(url_blanchett)
assert r_blanchett.status_code == 200, f"Erreur {r_blanchett.status_code}"

soup_blanchett = BeautifulSoup(r_blanchett.text, "html.parser")

selector_films = "#mw-content-text div ul:nth-of-type(3) li i a"
films_blanchett = soup_blanchett.select(selector_films)

films_data = [
    {
        "titre": film.attrs["title"],
        "url_wikipedia": url_wikipedia + film.attrs["href"]
    }
    for film in films_blanchett
    if not (
        film.attrs.get("class") == ["new"] # Film sans page
        or film.attrs["title"] == "Galadriel" # Mauvais lien
    )
]

films = pd.DataFrame(films_data)

films

Le sélecteur CSS que nous avons utilisé ne permettait pas d'obtenir la réponse à notre question car il ne capturait pas toutes les listes d'acteurs (organisation différente pour *Coffee and Cigarettes*, double colonne pour *Aviator*, ...). En effet, les pages Wikipedia des films ne sont pas uniformes et il n'est pas possible d'extraire la distribution de tous les films avec le même sélecteur.

Pour remédier à cela, nous proposons ici d'aller scraper la liste des acteurs sur le site [TMDB](https://www.themoviedb.org/) (*The Movie Database*) dont les pages obéissent toutes à la même organisation. Les pages Wikipedia relatives à des films contiennent toutes un lien externe vers ce site.

1. Pour chaque film, scraper la page Wikipedia pour récupérer le lien vers la page TMDB associée et déduire le lien du casting complet qui ser ajouté le dans une nouvelle colonne du dataframe `films`.

2. La liste des acteurs d'un film se présente comme une liste ordonnée `ol` dans les pages TMDB. Scraper les pages de casting pour ajouter la liste des acteurs de chaque film dans une nouvelle colonne du dataframe `films`.

3. Utiliser le résultat de la question précédente pour répondre à la question initiale : avec quels acteurs Cate Blanchett a-t-elle partagé l'affiche le plus souvent au cours des années 2000 ?